### Code to collect data for unemployment rate and job vacancy rate from Eurostat

In [3]:
import sdmx
import pandas as pd

import json
from typing import Any

import numpy as np
import matplotlib.pyplot as plt

In [15]:
# connect to Eurostat SDMX client
estat = sdmx.Client('ESTAT')

### Get an overview of available dataflows

In [5]:
flow = estat.dataflow()

In [6]:
flow

<sdmx.StructureMessage>
  <Header>
    id: 'DF1715331747'
    prepared: '2024-05-10T11:02:27.094000+02:00'
    sender: <Agency ESTAT>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (7601): MAR_GO_AM_LV MAR_GO_AM_ME MAR_GO_AM_MT MAR...
  DataStructureDefinition (7159): ESTAT:MAR_GO_AM_LV(23.2) ESTAT:MAR_GO...

In [7]:
dataflows = sdmx.to_pandas(flow.dataflow)

In [9]:
unemployment_data = dataflows[dataflows.str.contains("unemployment")]

In [11]:
# overview of available dataflows on the subject of unemployment
unemployment_data.head()

ENPE_LFSA_URGAN2                  Long-term unemployment rates by sex
TIPSLM70            Long-term unemployment rate, % of active popul...
TIPSLM80            Youth unemployment rate - % of active populati...
LFSQ_SUP_AGE        Supplementary indicators to unemployment by se...
LFSQ_SUP_EDU        Supplementary indicators to unemployment by se...
dtype: object

In [12]:
job_vacancy_data = dataflows[dataflows.str.contains("vacancy")]

In [14]:
# overview of available dataflows on the subject of unemployment
job_vacancy_data.head()

EI_LMJV_Q_R2                     Job vacancy rate - quarterly data
JVS_A_NACE1      Job vacancy statistics by occupation, NUTS 2 r...
JVS_A_NACE2      Job vacancy statistics by occupation, NUTS 2 r...
JVS_A_RATE_R2    Job vacancy rate by NACE Rev. 2 activity - ann...
JVS_Q_ISCO_R2    Job vacancy statistics by NACE Rev. 2 activity...
dtype: object

### Download data for unemployment rate and job vacancy rate from Eurostat

- job vacancy rate data code: JVS_Q_NACE2
- unemployment rate data code: UNE_RT_Q

In this version of the code, I downloaded data for four countries (Germany, Spain, the Netherlands and Sweden) to get an overview.
However, I only write about the case of Germany in the blog post.

In [16]:
data_msg = estat.data(
        "JVS_Q_NACE2",
        key=dict(
            freq="Q", s_adj="SA", nace_r2="B-S", sizeclas="TOTAL",
            indic_em="JOBRATE", geo="DE+SE+NL+ES"),
        params=dict(startPeriod="2010-Q4")
    )

In [185]:
data = data_msg.data

In [186]:
data_msg.response.headers['content-type']

'application/vnd.sdmx.genericdata+xml;version=2.1'

In [187]:
jvr = sdmx.to_pandas(data)

In [188]:
jvr

freq  s_adj  nace_r2  sizeclas  indic_em  geo  TIME_PERIOD
Q     SA     B-S      TOTAL     JOBRATE   DE   2010-Q4        2.1
                                               2011-Q1        2.4
                                               2011-Q2        2.2
                                               2011-Q3        2.3
                                               2011-Q4        2.3
                                                             ... 
                                          SE   2022-Q4        3.0
                                               2023-Q1        3.1
                                               2023-Q2        2.9
                                               2023-Q3        2.7
                                               2023-Q4        2.5
Name: value, Length: 212, dtype: float64

In [189]:
data_final = jvr.xs(
        key=("SA", "B-S", "TOTAL", "JOBRATE"),
        level=("s_adj", "nace_r2", "sizeclas", "indic_em")
    )

In [190]:
data_final

freq  geo  TIME_PERIOD
Q     DE   2010-Q4        2.1
           2011-Q1        2.4
           2011-Q2        2.2
           2011-Q3        2.3
           2011-Q4        2.3
                         ... 
      SE   2022-Q4        3.0
           2023-Q1        3.1
           2023-Q2        2.9
           2023-Q3        2.7
           2023-Q4        2.5
Name: value, Length: 212, dtype: float64

In [191]:
data_msg_un = estat.data("UNE_RT_Q", key=dict(freq="Q", s_adj="SA", sex="T", unit="PC_ACT", geo="DE+SE+NL+ES", age="Y15-74"), params=dict(startPeriod="2010-Q4"))

In [192]:
une = sdmx.to_pandas(data_msg_un.data)

In [193]:
une.head()

freq  s_adj  age     unit    sex  geo  TIME_PERIOD
Q     SA     Y15-74  PC_ACT  T    DE   2010-Q4        6.3
                                       2011-Q1        5.9
                                       2011-Q2        5.5
                                       2011-Q3        5.4
                                       2011-Q4        5.2
Name: value, dtype: float64

In [194]:
data_final_une = une.xs(
        key=("SA", "Y15-74", "PC_ACT", "T"),
        level=("s_adj", "age", "unit", "sex")
    )

In [195]:
data_final_une

freq  geo  TIME_PERIOD
Q     DE   2010-Q4        6.3
           2011-Q1        5.9
           2011-Q2        5.5
           2011-Q3        5.4
           2011-Q4        5.2
                         ... 
      SE   2022-Q4        7.4
           2023-Q1        7.4
           2023-Q2        7.5
           2023-Q3        7.9
           2023-Q4        8.0
Name: value, Length: 212, dtype: float64

In [196]:
data = pd.merge(data_final, data_final_une, left_index=True, right_on=("freq", "geo", "TIME_PERIOD"))
data.columns = ["job_vacancy_rate", "unemployment_rate"]

In [197]:
data.head()

job_vacancy_rate  unemployment_rate
freq geo TIME_PERIOD                                     
Q    DE  2010-Q4                   2.1                6.3
         2011-Q1                   2.4                5.9
         2011-Q2                   2.2                5.5
         2011-Q3                   2.3                5.4
         2011-Q4                   2.3                5.2

In [198]:
data.isna().sum()

job_vacancy_rate     0
unemployment_rate    0
dtype: int64

In [108]:
de_data = data.xs("DE", level="geo")

In [109]:
de_data

job_vacancy_rate  unemployment_rate
freq TIME_PERIOD                                     
Q    2010-Q4                   2.1                6.3
     2011-Q1                   2.4                5.9
     2011-Q2                   2.2                5.5
     2011-Q3                   2.3                5.4
     2011-Q4                   2.3                5.2
     2012-Q1                   2.2                5.2
     2012-Q2                   2.4                5.1
     2012-Q3                   2.3                5.1
     2012-Q4                   2.2                5.0
     2013-Q1                   2.0                5.1
     2013-Q2                   2.1                5.0
     2013-Q3                   2.2                4.8
     2013-Q4                   2.3                4.9
     2014-Q1                   2.2                4.9
     2014-Q2                   2.2                4.7
     2014-Q3                   2.3                4.6
     2014-Q4                   2.4                4.7
     2015-Q1                   2.4                4.5
     2015-Q2                   2.3                4.4
     2015-Q3                   2.3                4.2
     2015-Q4                   2.5                4.4
     2016-Q1                   2.4                4.1
     2016-Q2                   2.5                4.0
     2016-Q3                   2.4                3.9
     2016-Q4                   2.4                3.7
     2017-Q1                   2.6                3.7
     2017-Q2                   2.8                3.6
     2017-Q3                   2.8                3.5
     2017-Q4                   2.7                3.4
     2018-Q1                   2.9                3.3
     2018-Q2                   3.0                3.3
     2018-Q3                   3.1                3.2
     2018-Q4                   3.2                3.1
     2019-Q1                   3.3                3.0
     2019-Q2                   3.4                2.9
     2019-Q3                   3.3                3.0
     2019-Q4                   3.2                3.0
     2020-Q1                   2.9                3.3
     2020-Q2                   2.1                3.7
     2020-Q3                   2.4                4.0
     2020-Q4                   2.7                3.7
     2021-Q1                   2.9                4.0
     2021-Q2                   2.9                3.8
     2021-Q3                   3.4                3.4
     2021-Q4                   3.8                3.4
     2022-Q1                   4.2                3.1
     2022-Q2                   4.6                3.2
     2022-Q3                   4.3                3.2
     2022-Q4                   4.3                3.1
     2023-Q1                   4.1                3.0
     2023-Q2                   4.1                3.1
     2023-Q3                   4.1                3.1
     2023-Q4                   3.8                3.1

In [114]:
viz_data = []
for i in de_data.index:
    row = {}
    row["side"] = "top"
    row["date"] = i[1]
    row["vacancy_rate"] = de_data.loc[i]["job_vacancy_rate"]
    row["unemployment_rate"] = de_data.loc[i]["unemployment_rate"]
    viz_data.append(row)
    

In [199]:
def prepare_country_data(df: pd.DataFrame, country: str) -> list[dict[str: Any]]:
    data = df.xs(country, level="geo")
    viz_data = []
    for i, idx in enumerate(data.index):
        row = {}
        if i % 2 == 0:
            row["side"] = "top"
        row["date"] = idx[1]
        row["vacancy_rate"] = data.loc[i]["job_vacancy_rate"]
        row["unemployment_rate"] = data.loc[i]["unemployment_rate"]
        viz_data.append(row)

SyntaxError: invalid syntax (3429825278.py, line 9)

In [116]:
len(viz_data)

53

In [118]:
with open("de_data.json", "w") as f:
    json.dump(viz_data, f)

In [119]:
se_data = data.xs("SE", level="geo")

In [120]:
viz_data_se = []
for i in se_data.index:
    row = {}
    row["side"] = "top"
    row["date"] = i[1]
    row["vacancy_rate"] = se_data.loc[i]["job_vacancy_rate"]
    row["unemployment_rate"] = se_data.loc[i]["unemployment_rate"]
    viz_data_se.append(row)

In [121]:
viz_data_se

[{'side': 'top',
  'date': '2010-Q4',
  'vacancy_rate': 1.4,
  'unemployment_rate': 8.3},
 {'side': 'top',
  'date': '2011-Q1',
  'vacancy_rate': 1.4,
  'unemployment_rate': 8.1},
 {'side': 'top',
  'date': '2011-Q2',
  'vacancy_rate': 1.6,
  'unemployment_rate': 8.0},
 {'side': 'top',
  'date': '2011-Q3',
  'vacancy_rate': 1.6,
  'unemployment_rate': 7.8},
 {'side': 'top',
  'date': '2011-Q4',
  'vacancy_rate': 1.5,
  'unemployment_rate': 8.0},
 {'side': 'top',
  'date': '2012-Q1',
  'vacancy_rate': 1.6,
  'unemployment_rate': 7.9},
 {'side': 'top',
  'date': '2012-Q2',
  'vacancy_rate': 1.6,
  'unemployment_rate': 8.0},
 {'side': 'top',
  'date': '2012-Q3',
  'vacancy_rate': 1.5,
  'unemployment_rate': 8.3},
 {'side': 'top',
  'date': '2012-Q4',
  'vacancy_rate': 1.4,
  'unemployment_rate': 8.4},
 {'side': 'top',
  'date': '2013-Q1',
  'vacancy_rate': 1.5,
  'unemployment_rate': 8.3},
 {'side': 'top',
  'date': '2013-Q2',
  'vacancy_rate': 1.4,
  'unemployment_rate': 8.2},
 {'side': 

In [122]:
with open("se_data.json", "w") as f:
    json.dump(viz_data_se, f)

In [124]:
nl_data = data.xs("NL", level="geo")

In [125]:
viz_data_nl = []
for i in nl_data.index:
    row = {}
    row["side"] = "top"
    row["date"] = i[1]
    row["vacancy_rate"] = nl_data.loc[i]["job_vacancy_rate"]
    row["unemployment_rate"] = nl_data.loc[i]["unemployment_rate"]
    viz_data_nl.append(row)

In [126]:
with open("nl_data.json", "w") as f:
    json.dump(viz_data_nl, f)

In [127]:
nl_data

job_vacancy_rate  unemployment_rate
freq TIME_PERIOD                                     
Q    2010-Q4                   1.6                6.0
     2011-Q1                   1.7                5.9
     2011-Q2                   1.7                5.9
     2011-Q3                   1.6                6.0
     2011-Q4                   1.5                6.3
     2012-Q1                   1.5                6.5
     2012-Q2                   1.4                6.7
     2012-Q3                   1.3                6.9
     2012-Q4                   1.3                7.3
     2013-Q1                   1.2                7.7
     2013-Q2                   1.2                8.1
     2013-Q3                   1.2                8.5
     2013-Q4                   1.2                8.6
     2014-Q1                   1.3                8.7
     2014-Q2                   1.4                8.5
     2014-Q3                   1.5                8.2
     2014-Q4                   1.5                8.2
     2015-Q1                   1.6                8.1
     2015-Q2                   1.7                7.9
     2015-Q3                   1.7                7.8
     2015-Q4                   1.8                7.7
     2016-Q1                   1.9                7.4
     2016-Q2                   1.9                7.3
     2016-Q3                   2.0                6.8
     2016-Q4                   2.1                6.5
     2017-Q1                   2.3                6.3
     2017-Q2                   2.5                6.1
     2017-Q3                   2.6                5.7
     2017-Q4                   2.7                5.4
     2018-Q1                   2.8                5.2
     2018-Q2                   3.0                5.0
     2018-Q3                   3.1                4.7
     2018-Q4                   3.1                4.6
     2019-Q1                   3.2                4.6
     2019-Q2                   3.2                4.4
     2019-Q3                   3.3                4.3
     2019-Q4                   3.3                4.4
     2020-Q1                   2.5                4.2
     2020-Q2                   2.4                4.9
     2020-Q3                   2.5                5.3
     2020-Q4                   2.6                5.0
     2021-Q1                   2.9                4.8
     2021-Q2                   3.7                4.4
     2021-Q3                   4.1                3.9
     2021-Q4                   4.3                3.8
     2022-Q1                   4.9                3.6
     2022-Q2                   5.0                3.4
     2022-Q3                   4.9                3.6
     2022-Q4                   4.8                3.5
     2023-Q1                   4.7                3.6
     2023-Q2                   4.6                3.6
     2023-Q3                   4.4                3.5
     2023-Q4                   4.4                3.5

In [128]:
es_data = data.xs("ES", level="geo")

In [129]:
viz_data_es = []
for i in es_data.index:
    row = {}
    row["side"] = "top"
    row["date"] = i[1]
    row["vacancy_rate"] = es_data.loc[i]["job_vacancy_rate"]
    row["unemployment_rate"] = es_data.loc[i]["unemployment_rate"]
    viz_data_es.append(row)

In [130]:
with open("es_data.json", "w") as f:
    json.dump(viz_data_es, f)

In [15]:
data_msg_un = estat.data("LFSQ_UPGAL", key=dict(freq="Q", sex="T", unit="PC_ACT", geo="DE+SE+NL+ES", age="Y15-74"), params=dict(startPeriod="2010-Q4"))

HTTPError: 400 Client Error: Bad Request for url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/LFSQ_UPGAL/Q.PC_ACT.T.Y15-74.DE+ES+NL+SE?startPeriod=2010-Q4